In [1]:
import h5py
%matplotlib inline
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib

import sys
import shutil
import glob
import optparse
import os
import json
import cPickle as pkl
import pandas as pd
import numpy as np
import pylab as pl
import scipy.stats as stats
import seaborn as sns

import pprint
pp = pprint.PrettyPrinter(indent=4)
sys.path.append('/n/coxfs01/cechavarria/repos/2p-pipeline/')
from pipeline.python.paradigm import utils as util


/n/coxfs01/cechavarria/repos/2p-pipeline/pipeline/python/traces/utils.py:9: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/cesar/anaconda2/envs/pipeline/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/cesar/anaconda2/envs/pipeline/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/cesar/anaconda2/envs/pipeline/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/cesar/anaconda2/envs/pipeline/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/cesar/anaconda2/envs/p

In [8]:
class struct: pass

opts = struct()
opts.rootdir = '/n/coxfs01/2p-data'
opts.animalid = 'JC110'
opts.session = '20190909'
opts.acquisition = 'FOV1_zoom4p0x'
opts.traceid = 'traces102'
opts.combined_run = 'scenes_combined'
opts.run_list = ['scenes_run1','scenes_run2','scenes_run3','scenes_run4','scenes_run5','scenes_run6','scenes_run7','scenes_run8']

s2p = True

#Set up paths
acquisition_dir = os.path.join(opts.rootdir, opts.animalid, opts.session, opts.acquisition)

if s2p:
    combined_traceid_file_dir = os.path.join(acquisition_dir,opts.combined_run,'traces','%s_s2p'%(opts.traceid),'files')
else:
    combined_traceid_file_dir = os.path.join(acquisition_dir,opts.combined_run,'traces',opts.traceid,'files')
if not os.path.exists(combined_traceid_file_dir):
    os.makedirs(combined_traceid_file_dir)
    
combined_paradigm_dir = os.path.join(acquisition_dir,opts.combined_run,'paradigm','files')
if not os.path.exists(combined_paradigm_dir):
    os.makedirs(combined_paradigm_dir)

In [14]:
for run_idx,run in enumerate(opts.run_list):
    print(run)
# run_idx = 0
# run = 'scenes_run1'


    #% Set up paths:   
    if s2p:
        traceid_dir = os.path.join(acquisition_dir, run, 'traces', '%s_s2p'%(opts.traceid))
    else:
        traceid_dir = util.get_traceid_from_acquisition(acquisition_dir, run, opts.traceid)
    run_dir = traceid_dir.split('/traces')[0]
    trace_arrays_dir = os.path.join(traceid_dir,'files')
    paradigm_dir = os.path.join(acquisition_dir, run, 'paradigm')


    #read file
    parsedtraces_filepath = glob.glob(os.path.join(traceid_dir, 'files','parsedtraces*'))[0]
    file_grp = h5py.File(parsedtraces_filepath, 'r')



    curr_slice = 'Slice01'#hard-code planar data for now
    
    trial_fid = np.array(file_grp['Slice01']['trial_fid'])
    trial_run = np.ones(trial_fid.shape)*(run_idx+1)
    #get raw pixel value arrays
    pix_raw_run = np.array(file_grp[curr_slice]['traces']['raw'])
    pix_cell_run = np.array(file_grp[curr_slice]['traces']['np_subtracted'])
    pix_np_run = np.array(file_grp[curr_slice]['traces']['neuropil'])

    print(pix_raw_run.shape)



    #save attributes and stack traces
    if run_idx ==0:
         # Create outfile:
        combined_array_fn = 'parsedtraces.hdf5'
        combined_array_filepath = os.path.join(combined_traceid_file_dir, combined_array_fn)
        combined_grp = h5py.File(combined_array_filepath, 'w')
        
        combined_grp.attrs['source_dir'] = file_grp.attrs['source_dir']
        combined_grp.attrs['framerate'] = file_grp.attrs['framerate']
        combined_grp.attrs['volumerate'] = file_grp.attrs['volumerate']
        combined_grp.attrs['nvolumes'] = file_grp.attrs['nvolumes']
        combined_grp.attrs['nfiles'] = file_grp.attrs['nfiles']
        combined_grp.attrs['iti_pre'] = file_grp.attrs['iti_pre']
        combined_grp.attrs['iti_post'] = file_grp.attrs['iti_post']
        combined_grp.attrs['pre_frames'] = file_grp.attrs['pre_frames']
        combined_grp.attrs['post_frames'] = file_grp.attrs['post_frames'] 
        combined_grp.attrs['stim_frames'] = file_grp.attrs['stim_frames']
        combined_grp.attrs['nrois'] = pix_cell_run.shape[2]
        curr_tstamps = np.array(file_grp['Slice01']['frames_tsec'])
        tstamps_indices = np.array(file_grp['Slice01']['frames_tsec'])
        
        
        
        if 's2p_cell_rois' in file_grp.attrs.keys():
            combined_grp.attrs['s2p_cell_rois'] = file_grp.attrs['s2p_cell_rois']

        pix_raw_combo = pix_raw_run
        pix_cell_combo = pix_cell_run
        pix_np_combo = pix_np_run
        trial_fid_combo = trial_fid
        trial_run_combo = trial_run
    else:
        #for now, lop off timepoints if doesn't match what we already have, if volumerate fast enough. this is negligible
        if pix_raw_run.shape[1]>pix_raw_combo.shape[1]:
            extra_frames = pix_raw_run.shape[1]-pix_raw_combo.shape[1]
            pix_raw_run = pix_raw_run[:,:-extra_frames,:]
            pix_cell_run = pix_cell_run[:,:-extra_frames,:]
            pix_np_run = pix_np_run[:,:-extra_frames,:]
        
        pix_raw_combo = np.vstack((pix_raw_combo,pix_raw_run))
        pix_cell_combo = np.vstack((pix_cell_combo,pix_cell_run))
        pix_np_combo = np.vstack((pix_np_combo,pix_np_run))
        
        trial_fid_combo = np.hstack((trial_fid_combo,trial_fid))
        trial_run_combo = np.hstack((trial_run_combo,trial_run))
        
    file_grp.close()

#save combined traces to file
#save arrays to file

fset = combined_grp.create_dataset('/'.join([curr_slice, 'frames_tsec']), curr_tstamps.shape, curr_tstamps.dtype)
fset[...] = curr_tstamps

tset = combined_grp.create_dataset('/'.join([curr_slice, 'frames_indices']), tstamps_indices.shape, tstamps_indices.dtype)
tset[...] = tstamps_indices 

fidset = combined_grp.create_dataset('/'.join([curr_slice, 'trial_fid']), trial_fid_combo.shape, trial_fid_combo.dtype)
fidset[...] = trial_fid_combo

runset = combined_grp.create_dataset('/'.join([curr_slice, 'trial_run']), trial_run_combo.shape, trial_run_combo.dtype)
runset[...] = trial_run_combo

raw_combined = combined_grp.create_dataset('/'.join([curr_slice, 'traces', 'raw']), pix_raw_combo.shape, pix_raw_combo.dtype)
raw_combined[...] = pix_raw_combo

cell_combined = combined_grp.create_dataset('/'.join([curr_slice, 'traces', 'np_subtracted']), pix_cell_combo.shape, pix_cell_combo.dtype)
cell_combined[...] = pix_cell_combo

np_combined = combined_grp.create_dataset('/'.join([curr_slice, 'traces', 'neuropil']), pix_np_combo.shape, pix_np_combo.dtype)
np_combined[...] = pix_np_combo

combined_grp.close()

print('Done Combining Traces!')
print('Saved all info to: %s'%(combined_array_filepath))

#for run in runlist
for run_idx,run in enumerate(run_list):
    print(run)
# run = 'scenes_run1'
# run_idx = 0

    #open stimulus condition file
    stimconfig_fn = 'trial_conditions.hdf5'
    paradigm_dir = os.path.join(acquisition_dir, run, 'paradigm')
    stimconfig_filepath = os.path.join(paradigm_dir, 'files', stimconfig_fn)
    run_config_grp = h5py.File(stimconfig_filepath, 'r')

    config_run = np.array(run_config_grp['trial_config'])
    cond_run = np.array(run_config_grp['trial_cond'])
    img_run = np.array(run_config_grp['trial_img'])

    if run_idx ==0:
        #open output file
        combo_config_filepath = os.path.join(combined_paradigm_dir, stimconfig_fn)
        combo_config_grp = h5py.File(combo_config_filepath, 'w')

        combo_config_grp.attrs['ntrials'] = run_config_grp.attrs['ntrials']

        config_combo = config_run
        cond_combo = cond_run
        img_combo = img_run
    else:
        config_combo = np.hstack((config_combo,config_run))
        cond_combo = np.hstack((cond_combo,config_run))
        img_combo = np.hstack((img_combo,config_run))

    run_config_grp.close()

print('Combined Trial Conditions !')



#save arrays to file
imgcombo = combo_config_grp.create_dataset('trial_img', img_combo.shape, img_combo.dtype)
imgcombo[...] = img_combo

configcombo = combo_config_grp.create_dataset('trial_config', config_combo.shape, config_combo.dtype)
configcombo[...] = config_combo

condcombo = combo_config_grp.create_dataset('trial_cond', cond_combo.shape, cond_combo.dtype)
condcombo[...] = cond_combo

combo_config_grp.close()
print('Saved all info to: %s'%(combo_config_filepath))

scenes_run1
(300, 155, 178)
scenes_run2
(300, 155, 178)
scenes_run3
(300, 155, 178)
scenes_run4
(300, 155, 178)
scenes_run5
(300, 155, 178)
scenes_run6
(300, 155, 178)
Done Combining Traces!
Saved all info to: /n/coxfs01/2p-data/JC091/20190701/FOV1_zoom4p0x/scenes_combined/traces/traces001_s2p/files/parsedtraces.hdf5
scenes_run1
scenes_run2
scenes_run3
scenes_run4
scenes_run5
scenes_run6
Combined Trial Conditions !
Saved all info to: /n/coxfs01/2p-data/JC091/20190701/FOV1_zoom4p0x/scenes_combined/paradigm/files/trial_conditions.hdf5


In [15]:
trial_run

array([ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6

In [ ]:
config_run.shape

In [ ]:
run_config_grp.close()
combo_config_grp.close()

In [ ]:
pix_raw_combo.shape

In [ ]:
#combine paradigm info

In [ ]:
pix_raw_run.shape

In [ ]:
if pix_raw_run.shape[1]>pix_raw_combo.shape[1]:
    extra_frames = pix_raw_run.shape[1]-pix_raw_combo.shape[1]
    pix_raw_run = pix_raw_run[:,:-extra_frames,:]
    pix_cell_run = pix_cell_run[:,:-extra_frames,:]
    pix_np_run = pix_np_run[:,:-extra_frames,:]

In [ ]:
file_grp.close()
combined_grp.close()